Chapter5　自然言語処理と回帰型ニューラルネットワーク


In [0]:
!pip install --upgrade pip

In [0]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.4.0-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision
!pip3 install tqdm


In [0]:
import torch
torch.tensor([1,2,3]).to("cuda:0")

リスト5.1　全部で10000種類のトークンを20次元のベクトルで表現する場合

In [0]:
emb = nn.Embedding(10000, 20, padding_idx=0)
# Embedding層への入力はint64のTensor
inp = torch.tensor([1, 2, 5, 2, 10], dtype=torch.int64)
# 出力はfloat32のTensor
out = emb(inp)

Colaboratory における圧縮ファイルの展開

In [0]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar xf aclImdb_v1.tar.gz

ファイルの確認

In [0]:
!ls

In [0]:
!ls aclImdb

In [0]:
!ls aclImdb/test

In [0]:
!ls aclImdb/train

リスト5.2　関数の作成

In [0]:
import glob
import pathlib
import re

remove_marks_regex = re.compile("[,\.\(\)\[\]\*:;]|<.*?>")
shift_marks_regex = re.compile("([?!])")

def text2ids(text, vocab_dict):
    # !?以外の記号の削除
    text = remove_marks_regex.sub("", text)
    # !?と単語の間にスペースを挿入
    text = shift_marks_regex.sub(r" \1 ", text)
    tokens = text.split()
    return [vocab_dict.get(token, 0) for token in tokens]

def list2tensor(token_idxes, max_len=100, padding=True):
    if len(token_idxes) > max_len:
        token_idxes = token_idxes[:max_len]
    n_tokens = len(token_idxes)
    if padding:
        token_idxes = token_idxes \
            + [0] * (max_len - len(token_idxes))
    return torch.tensor(token_idxes, dtype=torch.int64), n_tokens

リスト5.3　Datasetクラスの作成

In [0]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, 
                              DataLoader,
                              TensorDataset)
import tqdm

In [0]:
class IMDBDataset(Dataset):
    def __init__(self, dir_path, train=True,
                 max_len=100, padding=True):
        self.max_len = max_len
        self.padding = padding
        
        path = pathlib.Path(dir_path)
        vocab_path = path.joinpath("imdb.vocab")
        
        # ボキャブラリファイルを読み込み、行ごとに分割
        self.vocab_array = vocab_path.open() \
                            .read().strip().splitlines()
        # 単語をキーとし、値がIDのdictを作る
        self.vocab_dict = dict((w, i+1) \
            for (i, w) in enumerate(self.vocab_array))
    
        if train:
            target_path = path.joinpath("train")
        else:
            target_path = path.joinpath("test")
        pos_files = sorted(glob.glob(
            str(target_path.joinpath("pos/*.txt"))))
        neg_files = sorted(glob.glob(
            str(target_path.joinpath("neg/*.txt"))))
        # posは1, negは0のlabelを付けて
        # (file_path, label)のtupleのリストを作成
        self.labeled_files = \
            list(zip([0]*len(neg_files), neg_files )) + \
            list(zip([1]*len(pos_files), pos_files))
  
    @property
    def vocab_size(self):
        return len(self.vocab_array)

    def __len__(self):
        return len(self.labeled_files)

    def __getitem__(self, idx):
        label, f = self.labeled_files[idx]
        # ファイルのテキストデータを読み取って小文字に変換
        data = open(f).read().lower()
        # テキストデータをIDのリストに変換
        data = text2ids(data, self.vocab_dict)
        # IDのリストをTensorに変換
        data, n_tokens = list2tensor(data, self.max_len, self.padding)
        return data, label, n_tokens

リスト5.4　訓練用とテスト用のDataLoaderの作成（your_path>を変更している）

In [0]:
train_data = IMDBDataset("aclImdb/")
test_data = IMDBDataset("aclImdb/", train=False)
train_loader = DataLoader(train_data, batch_size=32,
                          shuffle=True, num_workers=4)
test_loader = DataLoader(test_data, batch_size=32,
                        shuffle=False, num_workers=4)

リスト5.5　ネットワークの定義

In [0]:
class SequenceTaggingNet(nn.Module):
    def __init__(self, num_embeddings,
                 embedding_dim=50, 
                 hidden_size=50,
                 num_layers=1,
                 dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim,
                                padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim,
                            hidden_size, num_layers,
                            batch_first=True, dropout=dropout)
        self.linear = nn.Linear(hidden_size, 1)

    def forward(self, x, h0=None, l=None):
        # IDをEmbeddingで多次元のベクトルに変換する
        # xは(batch_size, step_size) 
        # -> (batch_size, step_size, embedding_dim)
        x = self.emb(x)
        # 初期状態h0と共にRNNにxを渡す
        # xは(batch_size, step_size, embedding_dim)
        # -> (batch_size, step_size, hidden_dim)
        x, h = self.lstm(x, h0)
        # 最後のステップのみ取り出す
        # xは(batch_size, step_size, hidden_dim)
        # -> (batch_size, 1)
        if l is not None:
            # 入力のもともとの長さがある場合はそれを使用する
            x = x[list(range(len(x))), l-1, :]
        else:
            # なければ単純に最後を使用する
            x = x[:, -1, :]
        # 取り出した最後のステップを線形層に入れる
        x = self.linear(x)
        # 余分な次元を削除する
        # (batch_size, 1) -> (batch_size, )
        x = x.squeeze()
        return x

リスト5.6　訓練の作成

In [0]:
def eval_net(net, data_loader, device="cpu"):
    net.eval()
    ys = []
    ypreds = []
    for x, y, l in data_loader:
        x = x.to(device)
        y = y.to(device)
        l = l.to(device)
        with torch.no_grad():
            y_pred = net(x, l=l)
            y_pred = (y_pred > 0).long()
            ys.append(y)
            ypreds.append(y_pred)
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    acc = (ys == ypreds).float().sum() / len(ys)
    return acc.item()

リスト5.7　評価の作成

In [0]:
from statistics import mean

# num_embeddingsには0を含めてtrain_data.vocab_size+1を入れる
net = SequenceTaggingNet(train_data.vocab_size+1, num_layers=2)
net.to("cuda:0")
opt = optim.Adam(net.parameters())
loss_f = nn.BCEWithLogitsLoss()

for epoch in range(10):
    losses = []
    net.train()
    for x, y, l in tqdm.tqdm(train_loader):
        x = x.to("cuda:0")
        y = y.to("cuda:0")
        l = l.to("cuda:0")
        y_pred = net(x, l=l)
        loss = loss_f(y_pred, y.float())
        net.zero_grad()
        loss.backward()
        opt.step()
        losses.append(loss.item())
    train_acc = eval_net(net, train_loader, "cuda:0")
    val_acc = eval_net(net, test_loader, "cuda:0")
    print(epoch, mean(losses), train_acc, val_acc)

リスト5.8　RNNを使用しないモデルの作成（<your_path>を変更している）

In [0]:
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression

train_X, train_y = load_svmlight_file(
    "aclImdb/train/labeledBow.feat")
test_X, test_y = load_svmlight_file(
    "aclImdb/test/labeledBow.feat",
    n_features=train_X.shape[1])

model = LogisticRegression(C=0.1, max_iter=1000)
model.fit(train_X, train_y)
model.score(train_X, train_y), model.score(test_X, test_y)

リスト5.9　PackedSequenceの性質を利用したモデルの作成

In [0]:
class SequenceTaggingNet2(SequenceTaggingNet):

    def forward(self, x, h0=None, l=None):
        # IDをEmbeddingで多次元のベクトルに変換
        x = self.emb(x)
        
        # 長さ情報が与えられている場合はPackedSequenceを作る
        if l is not None:
            x = nn.utils.rnn.pack_padded_sequence(
                x, l, batch_first=True)
        
        # RNNに通す
        x, h = self.lstm(x, h0)
        
        # 最後のステップを取り出して線形層に入れる
        if l is not None:
            # 長さ情報がある場合は最後の層の
            # 内部状態のベクトルを直接利用できる
            # LSTMは通常の内部状態の他にブロックセルの状態も
            # あるので内部状態のみを使用する
            hidden_state, cell_state = h
            x = hidden_state[-1]
        else:
            x = x[:, -1, :]
            
        # 線形層に入れる
        x = self.linear(x).squeeze()
        return x

リスト5.10　訓練部の作成

In [0]:
for epoch in range(10):
    losses = []
    net.train()
    for x, y, l in tqdm.tqdm(train_loader):
        # 長さの配列を長い順にソート
        l, sort_idx = torch.sort(l, descending=True)
        # 得られたインデクスを使用してx,yも並べ替え
        x = x[sort_idx]
        y = y[sort_idx]
        
        x = x.to("cuda:0")
        y = y.to("cuda:0")
        
        y_pred = net(x, l=l)
        loss = loss_f(y_pred, y.float())
        net.zero_grad()
        loss.backward()
        opt.step()
        losses.append(loss.item())
    train_acc = eval_net(net, train_loader, "cuda:0")
    val_acc = eval_net(net, test_loader, "cuda:0")
    print(epoch, mean(losses), train_acc, val_acc)

リスト5.11　語彙辞書と2つの変換関数の作成

In [0]:
# すべてのascii文字で辞書を作る
import string
all_chars = string.printable
vocab_size = len(all_chars)
vocab_dict = dict((c, i) for (i, c) in enumerate(all_chars))

# 文字列を数値のリストに変換する関数
def str2ints(s, vocab_dict):
    return [vocab_dict[c] for c in s]

# 数値のリストを文字列に変換する関数
def ints2str(x, vocab_array):
    return "".join([vocab_array[i] for i in x])

Colaboratory におけるファイルのアップロード

In [0]:
from google.colab import files

# ダイアログが表示され、ローカルのファイルを選択してアップロード
uploaded = files.upload()

リスト5.12　分割するDatasetクラスの定義

In [0]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, 
                              DataLoader,
                              TensorDataset)
import tqdm

In [0]:
class ShakespeareDataset(Dataset):
    def __init__(self, path, chunk_size=200):
        # ファイルを読み込み、数値のリストに変換する
        data = str2ints(open(path).read().strip(), vocab_dict)
        
        # Tensorに変換し、splitする
        data = torch.tensor(data, dtype=torch.int64).split(chunk_size)
        
        # 最後のchunkの長さをチェックして足りない場合には捨てる
        if len(data[-1]) < chunk_size:
            data = data[:-1]
            
        self.data = data
        self.n_chunks = len(self.data)
        
    def __len__(self):
        return self.n_chunks
    
    def __getitem__(self, idx):
        return self.data[idx]

リスト5.13　Datasetクラスを使用して、DataLoaderまでを作成（<your_path>を変更している）

In [0]:
ds = ShakespeareDataset("tinyshakespeare.txt", chunk_size=200)
loader = DataLoader(ds, batch_size=32, shuffle=True, num_workers=4)

リスト5.14　文章生成のモデル構築

In [0]:
class SequenceGenerationNet(nn.Module):
    def __init__(self, num_embeddings, 
                 embedding_dim=50, 
                 hidden_size=50,
                 num_layers=1, dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, 
                            hidden_size,
                            num_layers,
                            batch_first=True,
                            dropout=dropout)
        # Linerのoutputのサイズは最初のEmbeddingの
        # inputサイズと同じnum_embeddings
        self.linear = nn.Linear(hidden_size, num_embeddings)

    def forward(self, x, h0=None):
        x = self.emb(x)
        x, h = self.lstm(x, h0)
        x = self.linear(x)
        return x, h

リスト5.15　文章を生成する関数の作成

In [0]:
def generate_seq(net, start_phrase="The King said ",
                 length=200, temperature=0.8, device="cpu"):
    # モデルを評価モードにする
    net.eval()
    # 出力の数値を格納するリスト
    result = []
    
    # 開始文字列をTensorに変換
    start_tensor = torch.tensor(
        str2ints(start_phrase, vocab_dict),
        dtype=torch.int64
    ).to(device)
    # 先頭にbatch次元を付ける
    x0 = start_tensor.unsqueeze(0) 
    # RNNに通して出力と新しい内部状態を得る
    o, h = net(x0)
    # 出力を(正規化されていない)確率に変換
    out_dist = o[:, -1].view(-1).exp()
    # 確率から実際の文字のインデクスをサンプリング
    top_i = torch.multinomial(out_dist, 1)[0]
    # 結果を保存
    result.append(top_i)
    
    # 生成された結果を次々にRNNに入力していく
    for i in range(length):
        inp = torch.tensor([[top_i]], dtype=torch.int64)
        inp = inp.to(device)
        o, h = net(inp, h)
        out_dist = o.view(-1).exp()
        top_i = torch.multinomial(out_dist, 1)[0]
        result.append(top_i)
        
    # 開始文字列と生成された文字列をまとめて返す
    return start_phrase + ints2str(result, all_chars)

リスト5.16　文章を生成する関数の作成

In [0]:
from statistics import mean

net = SequenceGenerationNet(vocab_size, 20, 50,
                            num_layers=2, dropout=0.1)
net.to("cuda:0")
opt = optim.Adam(net.parameters())
# 多クラスの識別で問題なのでSoftmaxCrossEntropyLossが損失関数となる
loss_f = nn.CrossEntropyLoss()

for epoch in range(50):
    net.train()
    losses = []
    for data in tqdm.tqdm(loader):
        # xははじめから最後の手前の文字まで
        x = data[:, :-1]
        # yは2文字目から最後の文字まで
        y = data[:, 1:]

        x = x.to("cuda:0")
        y = y.to("cuda:0")

        y_pred, _ = net(x)
        # batchとstepの軸を統合してからCrossEntropyLossに渡す
        loss = loss_f(y_pred.view(-1, vocab_size), y.view(-1))
        net.zero_grad()
        loss.backward()
        opt.step()
        losses.append(loss.item())
    # 現在の損失関数と生成される文章の例を表示
    print(epoch, mean(losses))
    with torch.no_grad():
        print(generate_seq(net, device="cuda:0"))

Colaboratory における圧縮ファイルの展開

In [0]:
!wget http://www.manythings.org/anki/spa-eng.zip
!unzip spa-eng.zip

リスト5.17　補助関数の作成

In [0]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, 
                              DataLoader,
                              TensorDataset)
import tqdm

In [0]:
import re
import collections
import itertools

remove_marks_regex = re.compile(
    "[\,\(\)\[\]\*:;¿¡]|<.*?>")
shift_marks_regex = re.compile("([?!\.])")

unk = 0
sos = 1
eos = 2

def normalize(text):
    text = text.lower()
    # 不要な文字を除去
    text = remove_marks_regex.sub("", text)
    # ?!.と単語の間に空白を挿入
    text = shift_marks_regex.sub(r" \1", text)
    return text

def parse_line(line):
    line = normalize(line.strip())
    # 翻訳元(src)と翻訳先(trg)それぞれのトークンのリストを作る
    src, trg = line.split("\t")
    src_tokens = src.strip().split()
    trg_tokens = trg.strip().split()
    return src_tokens, trg_tokens

def build_vocab(tokens):
    # ファイル中のすべての文章でのトークンの出現数を数える
    counts = collections.Counter(tokens)
    # トークンの出現数の多い順に並べる
    sorted_counts = sorted(counts.items(), 
                           key=lambda c: c[1], reverse=True)
    # 3つのタグを追加して正引きリストと逆引き用辞書を作る
    word_list = ["<UNK>", "<SOS>", "<EOS>"] \
        + [x[0] for x in sorted_counts]
    word_dict = dict((w, i) for i, w in enumerate(word_list))
    return word_list, word_dict
    
def words2tensor(words, word_dict, max_len, padding=0):
    # 末尾に終了タグを付ける
    words = words + ["<EOS>"]
    # 辞書を利用して数値のリストに変換する
    words = [word_dict.get(w, 0) for w in words]
    seq_len = len(words)
    # 長さがmax_len以下の場合はパディングする
    if seq_len < max_len + 1:
        words = words + [padding] * (max_len + 1 - seq_len)
    # Tensorに変換して返す
    return torch.tensor(words, dtype=torch.int64), seq_len

リスト5.18　TranslationPairDatasetクラスの作成

In [0]:
class TranslationPairDataset(Dataset):
    def __init__(self, path, max_len=15):
        # 単語数が多い文章をフィルタリングする関数
        def filter_pair(p):
            return not (len(p[0]) > max_len 
                        or len(p[1]) > max_len)
        # ファイルを開き、パース/フィルタリングをする       
        with open(path) as fp:
            pairs = map(parse_line, fp)
            pairs = filter(filter_pair, pairs)
            pairs = list(pairs)
        # 文章のペアをソースとターゲットに分ける
        src = [p[0] for p in pairs]
        trg = [p[1] for p in pairs]
        #それぞれの語彙集を作成する
        self.src_word_list, self.src_word_dict = \
            build_vocab(itertools.chain.from_iterable(src))
        self.trg_word_list, self.trg_word_dict = \
            build_vocab(itertools.chain.from_iterable(trg))
        # 語彙集を使用してTensorに変換する
        self.src_data = [words2tensor(
            words, self.src_word_dict, max_len)
                for words in src]
        self.trg_data = [words2tensor(
            words, self.trg_word_dict, max_len, -100)
                for words in trg]
        
    def __len__(self):
        return len(self.src_data)
    
    def __getitem__(self, idx):
        src, lsrc = self.src_data[idx]
        trg, ltrg = self.trg_data[idx]
        return src, lsrc, trg, ltrg

リスト5.19　DatasetとDataLoaderの作成（<your_path>を変更している）

In [0]:
batch_size = 64
max_len = 10
path = "spa.txt"
ds = TranslationPairDataset(path, max_len=max_len)
loader = DataLoader(ds, batch_size=batch_size, shuffle=True,
                    num_workers=4)

リスト5.20　Encoderの作成

In [0]:
class Encoder(nn.Module):
    def __init__(self, num_embeddings,
                 embedding_dim=50, 
                 hidden_size=50,
                 num_layers=1,
                 dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim,
                                padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim,
                            hidden_size, num_layers,
                            batch_first=True, dropout=dropout)

    def forward(self, x, h0=None, l=None):
        x = self.emb(x)
        if l is not None:
            x = nn.utils.rnn.pack_padded_sequence(
                x, l, batch_first=True)
        _, h = self.lstm(x, h0)
        return h

リスト5.21　Decoderの作成（要再チェック）

In [0]:
class Decoder(nn.Module):
    def __init__(self, num_embeddings,
                 embedding_dim=50, 
                 hidden_size=50,
                 num_layers=1,
                 dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim,
                                padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_size,
                            num_layers, batch_first=True,
                            dropout=dropout)
        self.linear = nn.Linear(hidden_size, num_embeddings)
    def forward(self, x, h, l=None):
        x = self.emb(x)
        if l is not None:
            x = nn.utils.rnn.pack_padded_sequence(
                x, l, batch_first=True)
        x, h = self.lstm(x, h)
        if l is not None:
            x = nn.utils.rnn.pad_packed_sequence(x, batch_first=True, padding_value=0)[0]
        x = self.linear(x)
        return x, h

リスト5.22　翻訳する関数の作成

In [0]:
def translate(input_str, enc, dec, max_len=15, device="cpu"):
    # 入力文字列を数値化してTensorに変換
    words = normalize(input_str).split()
    input_tensor, seq_len = words2tensor(words, 
        ds.src_word_dict, max_len=max_len)
    input_tensor = input_tensor.unsqueeze(0)
    # Encoderで使用するので入力の長さもリストにしておく
    seq_len = [seq_len]
    # 開始トークンを準備
    sos_inputs = torch.tensor(sos, dtype=torch.int64)
    input_tensor = input_tensor.to(device)
    sos_inputs = sos_inputs.to(device)
    # 入力文字列をEncoderに入れてコンテキストを得る
    ctx = enc(input_tensor, l=seq_len)
    # 開始トークンとコンテキストをDecoderの初期値にセット
    z = sos_inputs
    h = ctx
    results = []
    for i in range(max_len):
        # Decoderで次の単語を予測
        o, h = dec(z.view(1, 1), h)
        # 線形層の出力が最も大きい場所が次の単語のID
        wi = o.detach().view(-1).max(0)[1]
        if wi.item() == eos:
            break
        results.append(wi.item())
        # 次の入力は今回の出力のIDを使用する
        z = wi
    # 記録しておいた出力のIDを文字列に変換
    return " ".join(ds.trg_word_list[i] for i in results)

リスト5.23　関数の動作の確認

In [0]:
enc = Encoder(len(ds.src_word_list), 100, 100, 2)
dec = Decoder(len(ds.trg_word_list), 100, 100, 2)
translate("I am a student.", enc, dec)

リスト5.24　オプティマイザーのパラメータ

In [0]:
enc = Encoder(len(ds.src_word_list), 100, 100, 2)
dec = Decoder(len(ds.trg_word_list), 100, 100, 2)
enc.to("cuda:0")
dec.to("cuda:0")
opt_enc = optim.Adam(enc.parameters(), 0.002)
opt_dec = optim.Adam(dec.parameters(), 0.002)
loss_f = nn.CrossEntropyLoss()

リスト5.25　モデルの学習部分（損失関数など）

In [0]:
from statistics import mean

def to2D(x):
    shapes = x.shape
    return x.reshape(shapes[0] * shapes[1], -1)

for epoc in range(30):
    # ネットワークを訓練モードにする
    enc.train(), dec.train()
    losses = []
    for x, lx, y, ly in tqdm.tqdm(loader):
        # xのPackedSequenceを作るために翻訳元の長さで降順にソート
        lx, sort_idx = lx.sort(descending=True)
        x, y, ly = x[sort_idx], y[sort_idx], ly[sort_idx]
        x, y = x.to("cuda:0"), y.to("cuda:0")
        # 翻訳元をEncoderに入れてコンテキストを得る
        ctx = enc(x, l=lx)
        
        # yのPackedSequenceを作るために翻訳先の長さで降順にソート
        ly, sort_idx = ly.sort(descending=True)
        y = y[sort_idx]
        # Decoderの初期値をセット
        h0 = (ctx[0][:, sort_idx, :], ctx[1][:, sort_idx, :])
        z = y[:, :-1].detach()
        # -100のままだとEmbeddingの計算でエラーが出てしまうので値を0に変更しておく
        z[z==-100] = 0
        # Decoderに通して損失関数を計算
        o, _ = dec(z, h0, l=ly-1)
        loss = loss_f(to2D(o[:]), to2D(y[:, 1:max(ly)]).squeeze())
        # Backpropagation（誤差逆伝播法）を実行
        enc.zero_grad(), dec.zero_grad()
        loss.backward()
        opt_enc.step(), opt_dec.step()
        losses.append(loss.item())
        
    # データセットに対して一通り計算したら現在の
    # 損失関数の値や翻訳結果を表示
    enc.eval(), dec.eval()
    print(epoc, mean(losses))
    with torch.no_grad():
        print(translate("I am a student.",
                         enc, dec, max_len=max_len, device="cuda:0"))
        print(translate("He likes to eat pizza.",
                         enc, dec, max_len=max_len, device="cuda:0"))
        print(translate("She is my mother.",
                         enc, dec, max_len=max_len, device="cuda:0"))